In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
from skimage import io, transform
import ast
import cv2

import tensorflow as tf
import PIL

from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from torchvision import models
vgg16 = models.vgg16_bn(pretrained=True)


plt.ion()  

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth



Using CUDA


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
% cd drive/My\ Drive/
# !ls

/content/drive/My Drive


In [ ]:
# Preparing the dataset
f=pd.read_csv("./covid-chestxray-dataset/metadata.csv")
keep_col = ['sex','age','finding','view','filename']
new_f = f[keep_col]
new_f.to_csv("./covid-chestxray-dataset/metadata_final.csv", index=False)

df = pd.read_csv("./covid-chestxray-dataset/metadata_final.csv")

df[df.finding != 'todo']
df[df.view != 'AP Erect'] # Not enough data
df = df.dropna()

listie = []
for index, row in df.iterrows():

    test_img = cv2.imread('./covid-chestxray-dataset/images/'+row['filename'])

    if test_img is None:
      print("CULPRIT")
      listie.append(row['filename'])

    text = row['finding']
    # print(text)
    text_split = text.split("/")

    if 'COVID-19' in text_split:
      df.loc[index,'finding'] = '[1,0]'
    else:
      df.loc[index,'finding'] = '[0,1]'
    
    if row['sex'] == 'M':
      df.loc[index,'sex'] = 0
    else:
      df.loc[index,'sex'] = 1

    df.loc[index,'age'] = row['age']/100

    if row['view'] == 'AP':
       df.loc[index,'view'] = '[1,0,0,0,0,0]'
    elif row['view'] == 'PA':
      df.loc[index,'view'] = '[0,1,0,0,0,0]'
    elif row['view'] == 'L':
      df.loc[index,'view'] = '[0,0,1,0,0,0]'
    elif row['view'] == 'Axial':
      df.loc[index,'view'] = '[0,0,0,1,0,0]'
    elif row['view'] == 'AP Supine':
      df.loc[index,'view'] = '[0,0,0,0,1,0]'
    elif row['view'] == 'Coronal':
      df.loc[index,'view'] = '[0,0,0,0,0,1]'
    # print(row['finding'])
    # print(row['sex'], row['age'])


CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT
CULPRIT


In [ ]:
for ii in listie:
  print(ii)
  df.drop(df.loc[df['filename']==ii].index, inplace=True)
df.to_csv('./covid-chestxray-dataset/metadata_final2.csv')

radiopaedia_org_covid-19-pneumonia-7_85703_0-dcm.nii.gz
radiopaedia_org_covid-19-pneumonia-14_85914_0-dcm.nii.gz
coronacases_org_001.nii.gz
coronacases_org_002.nii.gz
coronacases_org_003.nii.gz
coronacases_org_004.nii.gz
coronacases_org_005.nii.gz
coronacases_org_006.nii.gz
coronacases_org_007.nii.gz
coronacases_org_008.nii.gz
coronacases_org_009.nii.gz
coronacases_org_010.nii.gz
radiopaedia_org_covid-19-pneumonia-4_85506_1-dcm.nii.gz
radiopaedia_org_covid-19-pneumonia-29_86490_1-dcm.nii.gz
radiopaedia_org_covid-19-pneumonia-29_86491_1-dcm.nii.gz
radiopaedia_org_covid-19-pneumonia-23_86359_0-dcm.nii.gz
radiopaedia_org_covid-19-pneumonia-10_85902_1-dcm.nii.gz
radiopaedia_org_covid-19-pneumonia-10_85902_3-dcm.nii.gz
radiopaedia_org_covid-19-pneumonia-36_86526_0-dcm.nii.gz
radiopaedia_org_covid-19-pneumonia-27_86410_0-dcm.nii.gz
radiopaedia_org_covid-19-pneumonia-40_86625_0-dcm.nii.gz


In [ ]:
# Defining the required transformations for our custom data
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, sex, age, finding, view = sample['image'], sample['sex'], sample['age'], sample['finding'], sample['view']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        # h and w are swapped for landmarks because for images,
        # x and y axes are axis 1 and 0 respectively
        # landmarks = landmarks * [new_w / w, new_h / h]

        return {'image': img, 'sex': sex, 'age': age, 'finding':finding, 'view':view}


class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, sex, age, finding, view = sample['image'], sample['sex'], sample['age'], sample['finding'], sample['view']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        # landmarks = landmarks - [left, top]

        return {'image': image, 'sex': sex, 'age': age, 'finding':finding, 'view':view}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, sex, age, finding, view = sample['image'], sample['sex'], sample['age'], sample['finding'], sample['view']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        # print(image.shape)
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image).float(),
                'sex': torch.from_numpy(sex),
                'age': torch.from_numpy(age).float(),
                'finding': torch.from_numpy(finding),
                'view': torch.from_numpy(view)}

In [ ]:
# Loading the data from the folders

class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 5])
        # print(img_name)
        image = cv2.imread(img_name)
        # plt.imshow(image)
        # plt.show()
        sample = 0

        # print(image.shape)
        sex = self.landmarks_frame.iloc[idx,1]
        age = self.landmarks_frame.iloc[idx, 2]
        finding = self.landmarks_frame.iloc[idx, 3]
        view = self.landmarks_frame.iloc[idx, 4]
        view = ast.literal_eval(view)
        finding = ast.literal_eval(finding)

        sample = {'image': np.asarray(image), 'sex': np.asarray(sex), 'age': np.asarray(age), 'finding':np.asarray(finding), 'view':np.asarray(view)}

        if self.transform:
            sample = self.transform(sample)

        return sample




In [ ]:
# The original dataset
face_dataset = FaceLandmarksDataset(csv_file='./covid-chestxray-dataset/metadata_final2.csv',
                                    root_dir='./covid-chestxray-dataset/images')

# The transformed dataset
transformed_dataset = FaceLandmarksDataset(csv_file='./covid-chestxray-dataset/metadata_final2.csv',
                                    root_dir='./covid-chestxray-dataset/images',transform=transforms.Compose([
                                               Rescale(256),
                                               RandomCrop(224),
                                               ToTensor()
                                           ]))


In [ ]:
# Sanity check, we will check if our model is overfitting, we will be taking 4 images per class
train_set, val_set = torch.utils.data.random_split(transformed_dataset, [601, 67])
print(len(train_set))

overfit_trainset = []
overfit_valset = []
# output
counter = np.zeros((1,2))
pointer = -1
for i in range(len(train_set)):
  # print(counter)
  # print(train_set[i]['finding'])
  # print(int(train_set[i]['finding'].max(0, keepdim=True)[1]))
  if counter[0,0] == 11 and counter[0,1] == 11:
    break
  if (counter[0,int(train_set[i]['finding'].max(0, keepdim=True)[1])]<11):
    overfit_trainset.append(train_set[i])
    counter[0,int(train_set[i]['finding'].max(0, keepdim=True)[1])]+=1

# print("OVER")
# print(overfit_trainset)
counter = np.zeros((1,2))
pointer = -1
for i in range(len(val_set)):
  # print(val_set[i]['finding'])
  # print(int(val_set[i]['finding'].max(0, keepdim=True)[1]))
  # print(counter)
  if counter[0,0] == 6 and counter[0,1] == 6:
    break
  if (counter[0,int(val_set[i]['finding'].max(0, keepdim=True)[1])]<6):
    overfit_valset.append(val_set[i])
    counter[0,int(val_set[i]['finding'].max(0, keepdim=True)[1])]+=1


# print(overfit_valset)

601


In [ ]:
# Load the pretrained model from pytorch
vgg16 = models.vgg16_bn()
vgg16.load_state_dict(torch.load("/root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth", map_location="cuda:0"))

# Freeze training for all layers
for param in vgg16.features.parameters():
    param.requires_grad = False

vgg_extract = vgg16.features

class Feature_Extractor(nn.Module):
    def __init__(self):
        super(Feature_Extractor, self).__init__()
        self.extractor = vgg_extract
        
    def forward(self, x):
        x = self.extractor(x)
        # print(x.shape)
        # print(x.requires_grad)
        return x

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.fc1 = nn.Linear(25096, 4096, bias=False)
        self.fc2 = nn.Linear(4096, 4096, bias=True)
        self.fc3 = nn.Linear(4096, 128, bias=True)
        self.fc4 = nn.Linear(128, 2, bias=True)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        layer1 = F.relu(self.fc1(x))
        # print(layer1.requires_grad)
        layer2 = F.relu(self.fc2(self.dropout(layer1)))
        # print(layer2.requires_grad)
        layer3 = F.relu(self.fc3(self.dropout(layer1)))
        # print(layer3.requires_grad)
        layer4 = self.fc4(layer3)
        # print(layer4.requires_grad)
        return layer4


class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.fc1 = nn.Linear(25096, 4096, bias=False)
        self.fc2 = nn.Linear(4096, 4096, bias=True)
        self.fc3 = nn.Linear(4096, 128, bias=True)
        self.fc4 = nn.Linear(128, 2, bias=True)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        layer1 = F.relu(self.fc1(x))
        # print(layer1.requires_grad)
        layer2 = F.relu(self.fc2(self.dropout(layer1)))
        # print(layer2.requires_grad)
        layer3 = F.relu(self.fc3(self.dropout(layer1)))
        # print(layer3.requires_grad)
        layer4 = self.fc4(layer3)
        # print(layer4.requires_grad)
        return layer4

In [ ]:
model_extract = Feature_Extractor()
model_classifier = Classifier()

model_extract = model_extract.to("cuda")
model_classifier = model_classifier.to("cuda")

params_to_update = model_classifier.parameters()
# print("Params to learn:")
# for name,param in model_classifier.named_parameters():
#         if param.requires_grad == True:
#             print("\t",name)

epochs = 100
batch_sizey1 = 8
optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
dataloader_train = DataLoader(train_set, batch_size=batch_sizey1,
                        shuffle=True, num_workers=4)
dataloader_val = DataLoader(val_set, batch_size=batch_sizey1,
                        shuffle=True, num_workers=4)
# print("load")

count = 0
num_epoch_list = []

accuracy_train_list = []
loss_train_list = []

accuracy_val_list = []
loss_val_list = []
best_acc = 0
for j in range(epochs):
  num_epoch_list.append(j)
  count = 0
  loss_train = 0
  acc_train = 0
  for data in dataloader_train:
    # print("check")
    count+=1
    batch_sizey = data['finding'].shape[0]
    # Preprocessing the variables
    img = data['image']
    sex = data['sex']
    age = data['age']
    finding = data['finding']
    view = data['view']
    img = img.to("cuda")
    sex = sex.to("cuda")
    age = age.to("cuda")
    finding = finding.to("cuda")
    view = view.to("cuda")
    sex = torch.reshape(sex, (batch_sizey, 1))
    age = torch.reshape(age, (batch_sizey, 1))
    finding = torch.reshape(finding, (batch_sizey, 2))

    # Zero the parameter gradients
    optimizer.zero_grad()

    # Forward pass
    output = model_extract(img)
    flat_output = output.view(-1, 512* 7*7)
    x_final = torch.cat((flat_output,sex,age,view),1)
    output_final = model_classifier(x_final)
    find = finding.max(1, keepdim=True)[1]
    find = torch.reshape(find,(1,batch_sizey))
    find = find.flatten().to("cuda")
    loss = criterion(output_final,find)
    loss.backward()               # backward pass (compute parameter updates)
    optimizer.step()

    output_final = output_final.max(1, keepdim=True)[1]
    output_final = torch.reshape(output_final,(1,batch_sizey))

    loss_train += loss.data
    acc_train += torch.sum(output_final == find)

  avg_loss = loss_train  / 601
  avg_acc = acc_train  / 601
  accuracy_train_list.append(avg_acc)
  loss_train_list.append(avg_loss)
  # For validation

  count = 0
  loss_val = 0
  acc_val = 0
  for data in dataloader_val:
    count+=1
    batch_sizey = data['finding'].shape[0]
    # Preprocessing the variables
    img = data['image']
    sex = data['sex']
    age = data['age']
    finding = data['finding']
    view = data['view']
    img = img.to("cuda")
    sex = sex.to("cuda")
    age = age.to("cuda")
    finding = finding.to("cuda")
    view = view.to("cuda")
    sex = torch.reshape(sex, (batch_sizey, 1))
    age = torch.reshape(age, (batch_sizey, 1))
    finding = torch.reshape(finding, (batch_sizey, 2))

    # Forward pass
    output = model_extract(img)
    flat_output = output.view(-1, 512* 7*7)
    x_final = torch.cat((flat_output,sex,age,view),1)
    output_final = model_classifier(x_final)
    find = finding.max(1, keepdim=True)[1]
    find = torch.reshape(find,(1,batch_sizey))
    find = find.flatten().to("cuda")

    output_final = output_final.max(1, keepdim=True)[1]
    output_final = torch.reshape(output_final,(1,batch_sizey))

    loss_val += loss.data
    acc_val += torch.sum(output_final == find)

  print()
  print("EPOCH NUMBER: %d"%(j))
  print("Avg loss (train): {:.4f}".format(avg_loss))
  print("Avg acc (train): {:.4f}".format(avg_acc))

  avg_loss = loss_val  / 67
  avg_acc = acc_val  / 67
  accuracy_val_list.append(avg_acc)
  loss_val_list.append(avg_loss)

  if avg_acc>best_acc:
    print("saved")
    PATH = './no_dropout.pth'
    torch.save(model_classifier.state_dict(), PATH)
    best_acc = avg_acc
  print("Avg loss (val): {:.4f}".format(avg_loss))
  print("Avg acc (val): {:.4f}".format(avg_acc))


EPOCH NUMBER: 0
Avg loss (train): 0.0842
Avg acc (train): 0.5874
saved
Avg loss (val): 0.0621
Avg acc (val): 0.6716

EPOCH NUMBER: 1
Avg loss (train): 0.0791
Avg acc (train): 0.6456
Avg loss (val): 0.0526
Avg acc (val): 0.6716

EPOCH NUMBER: 2
Avg loss (train): 0.0726
Avg acc (train): 0.7155
saved
Avg loss (val): 0.0456
Avg acc (val): 0.7015

EPOCH NUMBER: 3
Avg loss (train): 0.0666
Avg acc (train): 0.7521
saved
Avg loss (val): 0.1311
Avg acc (val): 0.7164

EPOCH NUMBER: 4
Avg loss (train): 0.0606
Avg acc (train): 0.7870
Avg loss (val): 0.0112
Avg acc (val): 0.6866

EPOCH NUMBER: 5
Avg loss (train): 0.0536
Avg acc (train): 0.8236
Avg loss (val): 0.0524
Avg acc (val): 0.7015

EPOCH NUMBER: 6
Avg loss (train): 0.0479
Avg acc (train): 0.8469
Avg loss (val): 0.0157
Avg acc (val): 0.7164

EPOCH NUMBER: 7
Avg loss (train): 0.0416
Avg acc (train): 0.8602
saved
Avg loss (val): 0.0313
Avg acc (val): 0.7313

EPOCH NUMBER: 8
Avg loss (train): 0.0429
Avg acc (train): 0.8586
Avg loss (val): 0.0307

In [ ]:
plt.plot(num_epoch_list,accuracy_train_list, label='Train')
plt.plot(num_epoch_list,accuracy_val_list, label='Val')
plt.legend()
plt.title("Epoch vs Accuracy")
plt.xlabel("No. of Epochs")
plt.ylabel("Accuracy")
plt.show()
plt.plot(num_epoch_list,loss_train_list, label='Train')
plt.plot(num_epoch_list,loss_val_list, label='Val')
plt.legend()
plt.title("Epoch vs Loss")
plt.xlabel("No. of Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
|count = 1

# for i in train_set:
  # print(count)
  # print(i)
  # count+=1
dataloader_train = DataLoader(train_set, batch_size=8,
                        shuffle=True, num_workers=4)
dataloader_val = DataLoader(val_set, batch_size=8,
                        shuffle=True, num_workers=4)
dataloader_test = DataLoader(test_set, batch_size=8,
                        shuffle=True, num_workers=4)

print(len(dataloader_train))

67


In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(input)
target = torch.empty(3, dtype=torch.long).random_(5)
print(target)
output = loss(input, target)
output.backward()

tensor([[ 0.0245,  1.1838,  1.4191,  1.2461,  0.0910],
        [ 0.4712, -1.4013,  0.6251, -0.4971,  0.6933],
        [-0.6946,  0.5746,  1.3383, -0.5251,  1.0091]], requires_grad=True)
tensor([1, 1, 3])


In [ ]:
images_orig = []
directory = './data_new/covid'
for filename in os.listdir(directory):
    # if filename.endswith(".jpg") or filename.endswith(".png"):
        images_orig.append([filename,0])

directory = './data_new/not_covid'
for filename in os.listdir(directory):
    # if filename.endswith(".jpg") or filename.endswith(".png"):
        images_orig.append([filename,1])
print(images_orig)


In [ ]:
import shutil
images_orig = np.asarray(images_orig)
np.random.shuffle(images_orig)
total_len = images_orig.shape[0]
train_size = int(0.8*total_len)
val_size = int(0.1*total_len)
test_size = total_len - train_size - val_size

print(train_size,val_size,test_size,total_len)

for i in range(train_size):
  # print(images_orig[i])
  if images_orig[i,1] == '0':
    shutil.copyfile('./data_new/covid/'+images_orig[i,0], './data_new/train/covid/'+images_orig[i,0])
  else:
    shutil.copyfile('./data_new/not_covid/'+images_orig[i,0], './data_new/train/not_covid/'+images_orig[i,0])

for i in range(train_size,train_size+val_size):
  if images_orig[i,1] == '0':
    shutil.copyfile('./data_new/covid/'+images_orig[i,0], './data_new/val/covid/'+images_orig[i,0])
  else:
    shutil.copyfile('./data_new/not_covid/'+images_orig[i,0], './data_new/val/not_covid/'+images_orig[i,0])

for i in range(train_size+val_size,train_size+val_size+test_size):
  if images_orig[i,1] == '0':
    shutil.copyfile('./data_new/covid/'+images_orig[i,0], './data_new/test/covid/'+images_orig[i,0])
  else:
    shutil.copyfile('./data_new/not_covid/'+images_orig[i,0], './data_new/test/not_covid/'+images_orig[i,0])

676 84 86 846


In [ ]:
data_dir = './data_new'
TRAIN = 'train'
VAL = 'val'
TEST = 'test'

# VGG-16 Takes 224x224 images as input, so we resize all of them
data_transforms = {
    TRAIN: transforms.Compose([
        # Data augmentation is a good practice for the train set
        # Here, we randomly crop the image to 224x224 and
        # randomly flip it horizontally. 
        transforms.RandomResizedCrop(224),
        torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
        transforms.ToTensor(),
    ]),
    VAL: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
    TEST: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
}

image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x), 
        transform=data_transforms[x]
    )
    for x in [TRAIN, VAL, TEST]
}

dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=8,
        shuffle=True, num_workers=4
    )
    for x in [TRAIN, VAL, TEST]
}

dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}

for x in [TRAIN, VAL, TEST]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))
    
print("Classes: ")
class_names = image_datasets[TRAIN].classes
print(image_datasets[TRAIN].classes)

Loaded 676 images under train
Loaded 84 images under val
Loaded 86 images under test
Classes: 
['covid', 'not_covid']


In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    # plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# Get a batch of training data
# inputs, sex, age, finding, view = next(iter(dataloader_test))
# show_databatch(inputs, finding)
# testie = next(iter(dataloader_test))
# print(testie)
print(next(iter(dataloader_test)))

{'image': tensor([[[[0.0000, 0.0027, 0.0078,  ..., 0.1895, 0.2152, 0.2344],
          [0.0189, 0.0196, 0.0228,  ..., 0.1647, 0.1890, 0.2074],
          [0.0275, 0.0314, 0.0353,  ..., 0.1747, 0.1747, 0.1871],
          ...,
          [0.7765, 0.7744, 0.7701,  ..., 0.7255, 0.7333, 0.7373],
          [0.7765, 0.7765, 0.7703,  ..., 0.7324, 0.7333, 0.7373],
          [0.7736, 0.7743, 0.7765,  ..., 0.7265, 0.7355, 0.7451]],

         [[0.0000, 0.0027, 0.0078,  ..., 0.1895, 0.2152, 0.2344],
          [0.0189, 0.0196, 0.0228,  ..., 0.1647, 0.1890, 0.2074],
          [0.0275, 0.0314, 0.0353,  ..., 0.1747, 0.1747, 0.1871],
          ...,
          [0.7765, 0.7744, 0.7701,  ..., 0.7255, 0.7333, 0.7373],
          [0.7765, 0.7765, 0.7703,  ..., 0.7324, 0.7333, 0.7373],
          [0.7736, 0.7743, 0.7765,  ..., 0.7265, 0.7355, 0.7451]],

         [[0.0000, 0.0027, 0.0078,  ..., 0.1895, 0.2152, 0.2344],
          [0.0189, 0.0196, 0.0228,  ..., 0.1647, 0.1890, 0.2074],
          [0.0275, 0.0314, 0.035

In [ ]:
def visualize_model(vgg, num_images=6):
    was_training = vgg.training
    
    # Set model for evaluation
    vgg.train(False)
    vgg.eval() 
    
    images_so_far = 0

    for i, data in enumerate(dataloaders[TEST]):
        inputs, labels = data
        size = inputs.size()[0]
        
        if use_gpu:
            inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
        else:
            inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
        
        outputs = vgg(inputs)
        
        _, preds = torch.max(outputs.data, 1)
        predicted_labels = [preds[j] for j in range(inputs.size()[0])]
        
        print("Ground truth:")
        show_databatch(inputs.data.cpu(), labels.data.cpu())
        print("Prediction:")
        show_databatch(inputs.data.cpu(), predicted_labels)
        
        del inputs, labels, outputs, preds, predicted_labels
        torch.cuda.empty_cache()
        
        images_so_far += size
        if images_so_far >= num_images:
            break
        
    vgg.train(mode=was_training) # Revert model back to original training state

In [ ]:
def eval_model(vgg, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    
    test_batches = len(dataloader_test)
    print("Evaluating model")
    print('-' * 10)
    
    final_classes = []
    yhat_classes = []
    for i, data in enumerate(dataloader_test):
        # print(i,data[1])
        # print(data)
        for ii in data['finding']:
          final_classes.append(int(ii))
        if i % 100 == 0:
            print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)

        vgg.train(False)
        vgg.eval()
        inputs = data['image']
        sex = data['sex']
        age = data['age']
        finding = data['finding']
        view = data['view']

        if use_gpu:
            inputs, finding = Variable(inputs.cuda(), volatile=True), Variable(finding.cuda(), volatile=True)
        else:
            inputs, finding = Variable(inputs, volatile=True), Variable(finding, volatile=True)

        outputs = vgg(inputs)
        # print("original")
        # print(final_classes)
        # print(outputs)
        # preds = np.argmax(np.asarray(outputs),axis=1)
        _, preds = torch.max(outputs.data, 1)
        # print(preds)
        for ii in preds:
          yhat_classes.append(int(ii))

        loss = criterion(outputs, finding)

        loss_test += loss.data
        acc_test += torch.sum(preds == finding)

        del inputs, finding, outputs, preds
        torch.cuda.empty_cache()
        
    avg_loss = loss_test / test_batches
    avg_acc = acc_test / test_batches
    print(len(final_classes),len(yhat_classes))
    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Avg loss (test): {:.4f}".format(avg_loss))
    print("Avg acc (test): {:.4f}".format(avg_acc))
    print('-' * 10)
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(final_classes, yhat_classes)
    print('Accuracy (test): %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(final_classes, yhat_classes)
    print('Precision (test): %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(final_classes, yhat_classes)
    print('Recall (test): %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(final_classes, yhat_classes)
    print('F1 score (test): %f' % f1)


In [ ]:
# Load the pretrained model from pytorch
vgg16 = models.vgg16_bn()
vgg16.load_state_dict(torch.load("/root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth", map_location="cuda:0"))
print(vgg16.classifier[6].out_features) # 1000 


# Freeze training for all layers
for param in vgg16.features.parameters():
    param.require_grad = False

# Newly created modules have require_grad=True by default
# print(vgg16.features[43].out_features)
num_features = vgg16.classifier[6].in_features
num_features1 = vgg16.classifier[0].in_features
print(num_features1)
features = list(vgg16.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features,2)]) # Add our layer with 2 outputs
vgg16.classifier = nn.Sequential(*features) # Replace the model classifier
print(vgg16)

if use_gpu:
    print("checkie")
    vgg16.cuda() #.cuda() will move everything to the GPU side

1000
25088
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv

In [ ]:
# model_ft = models.vgg16(pretrained=True)

# for param in model_ft.parameters():
    # param.requires_grad = False

class Vgg_added_features(nn.Module):
    def __init__(self, originalModel):
        super(Vgg_added_features, self).__init__()
        self.features = nn.Sequential(*list(originalModel.features))
        self.poolie = originalModel.avgpool.cuda()
        self.classifier = nn.Sequential(nn.Linear(25096, 4096),originalModel.classifier[1:]).cuda()
        # self.classifier = nn.Sequential(originalModel.classifier[0:])
        #self.avg_pool = nn.AdaptiveAvgPool2d((7,7))
    
    def forward(self, x, x_cat):
        # print("INPUT SHAPE")
        # print(x.shape)
        x = self.features(x)
        # print("VGG16")
        # print(x)
        # print(x.shape)
        x = self.poolie(x)
        # print("MAXPOOL")
        # print(x.shape)
        # print(x)

        x= torch.reshape(x,(8,25088))

        # print("AFTER RESHAPING")
        # print(x.shape)
        # print(x_cat.shape)

        x_final = torch.cat((x,x_cat),1)
        # print("CHECK")
        x_final = torch.tensor(x_final).cuda()
        # x_final = Variable(x_final.cuda()),
        # x_final = torch.from_numpy(x_final)
        x_final = torch.tensor(self.classifier(x_final)).cuda()

        # x = torch.mul(torch.sign(x),torch.sqrt(torch.abs(x)+1e-12))
        # print(x.shape)
        # x = F.normalize(x, p=2, dim=1)
        # print(x.shape)
        # x = self.classifier(x)
        # print(x.shape)
        return x_final

model = Vgg_added_features(vgg16)
print(model)

Vgg_added_features(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): 

In [ ]:
def train_model(m_vgg, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(vgg16.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    
    train_batches = len(dataloader_train)
    val_batches = len(dataloader_val)
    test_batches = len(dataloader_test)
    print(train_batches,val_batches,test_batches)

    num_epoch_list = []

    accuracy_test_list = []
    loss_test_list = []

    accuracy_train_list = []
    loss_train_list = []

    accuracy_val_list = []
    loss_val_list = []
    for epoch in range(num_epochs):
        num_epoch_list.append(int(epoch))
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        loss_train = 0
        loss_val = 0
        loss_test = 0
        acc_train = 0
        acc_val = 0
        acc_test = 0
        
        m_vgg.train(True)
        count=1
        for i, data in enumerate(dataloader_train):
            # print(count)
            # count+=1
            if i % 100 == 0:
                print("\rTraining batch {}/{}".format(i, train_batches / 2), end='', flush=True)
                
            # Use half training dataset
            if i >= train_batches / 2:
                break
                
            # inputs, labels = data
            inputs = data['image']
            sex = data['sex']
            age = data['age']
            finding = data['finding']
            view = data['view']
            # finding = tf.cast(finding, tf.float64)
            finding = finding.float()
            sex = torch.reshape(sex, (8, 1))
            age = torch.reshape(age, (8, 1))
            final_cat = torch.cat((sex, age), 1)
            final_cat1 = torch.cat((final_cat,view),1)
            if 1:
                # print("check")
                inputs, finding,final_cat1 = Variable(inputs.cuda(),requires_grad=True), Variable(finding.cuda(),requires_grad=True), Variable(final_cat1.cuda(),requires_grad=True)
            else:
                inputs, finding,final_cat1 = Variable(inputs,requires_grad=True), Variable(finding,requires_grad=True), Variable(final_cat1,requires_grad=True)
            
            optimizer.zero_grad()
            
            outputs = m_vgg(inputs,final_cat1)
            # print("OUTPUTS OF VGG")
            # print(outputs.shape)
            _, preds = torch.max(outputs.data, 1)
            # finding = tf.cast(finding, tf.int32)
            finding = finding.long()
            loss = criterion(outputs, finding)
            print("done")
            
            loss.backward()
            optimizer.step()
            
            loss_train += loss.data
            acc_train += torch.sum(preds == finding)
            # print("THE accuracy at each step")
            # print(torch.sum(preds == finding))
            del inputs, finding, outputs, preds
            torch.cuda.empty_cache()
        
        print()
        # * 2 as we only used half of the dataset
        avg_loss = loss_train * 2 / (train_batches*8)
        avg_acc = acc_train * 2 / (train_batches*8)
        
        # For validation

        m_vgg.train(False)
        m_vgg.eval()
        count = 0
        for i, data in enumerate(dataloader_val):
            # count+=1
            # print(count)
            if i % 100 == 0:
                print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)
            if i >= 15:
                break   
            inputs = data['image']
            sex = data['sex']
            age = data['age']
            finding = data['finding']
            # finding = tf.cast(finding, tf.float64)
            finding = finding.float()
            view = data['view']
            
            sex = torch.reshape(sex, (8, 1))
            age = torch.reshape(age, (8, 1))
            final_cat = torch.cat((sex, age), 1)
            final_cat1 = torch.cat((final_cat,view),1)
            if 1:
                # print("check")
                inputs, finding,final_cat1 = Variable(inputs.cuda(),requires_grad=True), Variable(finding.cuda(),requires_grad=True), Variable(final_cat1.cuda(),requires_grad=True)
            else:
                inputs, finding,final_cat1 = Variable(inputs,requires_grad=True), Variable(finding,requires_grad=True), Variable(final_cat1,requires_grad=True)
            
            optimizer.zero_grad()
            
            outputs = m_vgg(inputs,final_cat1)
            
            _, preds = torch.max(outputs.data, 1)
            finding = finding.long()
            loss = criterion(outputs, finding)
            
            loss_val += loss.data
            acc_val += torch.sum(preds == finding)
            
            del inputs, finding, outputs, preds
            torch.cuda.empty_cache()

        avg_loss_val = loss_val / 67
        avg_acc_val = acc_val / 67

        # For test
        m_vgg.train(False)
        m_vgg.eval()
        count=0   
        for i, data in enumerate(dataloader_test):
            # count+=1
            # print(count)
            if i % 100 == 0:
                print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)
            # if i >= 15:
            #     break    
            inputs = data['image']
            sex = data['sex']
            age = data['age']
            finding = data['finding']
            view = data['view']
            # finding = tf.cast(finding, tf.float64)
            finding = finding.float()
            sex = torch.reshape(sex, (8, 1))
            age = torch.reshape(age, (8, 1))
            final_cat = torch.cat((sex, age), 1)
            final_cat1 = torch.cat((final_cat,view),1)
            if 1:
                # print("check")
                inputs, finding,final_cat1 = Variable(inputs.cuda(),requires_grad=True), Variable(finding.cuda(),requires_grad=True), Variable(final_cat1.cuda(),requires_grad=True)
            else:
                inputs, finding,final_cat1 = Variable(inputs,requires_grad=True), Variable(finding,requires_grad=True), Variable(final_cat1,requires_grad=True)
            
            optimizer.zero_grad()
            
            outputs = m_vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            finding = tf.cast(finding, tf.float64)
            finding = finding.long()
            loss = criterion(outputs, finding)
            print("done")
            
            loss_test += loss.data
            acc_test += torch.sum(preds == finding)
            
            del inputs, finding, outputs, preds
            torch.cuda.empty_cache()
        
        avg_loss_test = loss_test/ 66
        avg_acc_test = acc_test / 66
        
        print()
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        accuracy_train_list.append(avg_acc)
        loss_train_list.append(avg_loss)
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        accuracy_val_list.append(avg_acc_val)
        loss_val_list.append(avg_loss_val)
        print("Avg loss (test): {:.4f}".format(avg_loss_test))
        print("Avg acc (test): {:.4f}".format(avg_acc_test))
        accuracy_test_list.append(avg_acc_test)
        loss_test_list.append(avg_loss_test)
        print('-' * 10)
        print()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(vgg.state_dict())
        
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    vgg.load_state_dict(best_model_wts)
    return vgg, accuracy_test_list, loss_test_list, accuracy_train_list, loss_train_list,accuracy_val_list, loss_val_list,num_epoch_list

In [ ]:
modified_vgg = Vgg_added_features(vgg16)
if use_gpu:
    print("checkie")
    modified_vgg.cuda()
    
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)




checkie


In [ ]:
modified_vgg = Vgg_added_features(vgg16)
vgg16, accuracy_test, loss_test, accuracy_train, loss_train,accuracy_val, loss_val,num_epoch = train_model(modified_vgg, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=1)

67 9 9
Epoch 0/1
----------
Training batch 0/33.5done


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


RuntimeError: ignored

In [ ]:
# If you want to train the model for more than 2 epochs, set this to True after the first run
resume_training = False

if resume_training:
    print("Loading pretrained model..")
    vgg16.load_state_dict(torch.load('./VGG16_v2-OCT_Retina_half_dataset.pt'))
    print("Loaded!")

Loading pretrained model..


NameError: ignored

In [ ]:
if use_gpu:
    vgg16.cuda() #.cuda() will move everything to the GPU side
    
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

vgg16, accuracy_test, loss_test, accuracy_train, loss_train,accuracy_val, loss_val,num_epoch = train_model(vgg16, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=1)

In [ ]:
print("Test before training")
eval_model(vgg16, criterion)

Test before training
Evaluating model
----------
Test batch 0/9

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


66 66

Evaluation completed in 1m 7s
Avg loss (test): 0.6844
Avg acc (test): 3.3333
----------
Accuracy (test): 0.454545
Precision (test): 0.395349
Recall (test): 0.629630
F1 score (test): 0.485714


In [ ]:
def train_model(vgg, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(vgg.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    
    train_batches = len(dataloaders[TRAIN])
    val_batches = len(dataloaders[VAL])
    test_batches = len(dataloaders[TEST])

    num_epoch_list = []

    accuracy_test_list = []
    loss_test_list = []

    accuracy_train_list = []
    loss_train_list = []

    accuracy_val_list = []
    loss_val_list = []
    for epoch in range(num_epochs):
        num_epoch_list.append(int(epoch))
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        loss_train = 0
        loss_val = 0
        loss_test = 0
        acc_train = 0
        acc_val = 0
        acc_test = 0
        
        vgg.train(True)
        
        for i, data in enumerate(dataloaders[TRAIN]):
            if i % 100 == 0:
                print("\rTraining batch {}/{}".format(i, train_batches / 2), end='', flush=True)
                
            # Use half training dataset
            if i >= train_batches / 2:
                break
                
            inputs, labels = data
            
            if use_gpu:
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)
            
            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            print("OUTPUTS OF VGG")
            print(outputs.shape)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            loss_train += loss.data
            acc_train += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        print()
        # * 2 as we only used half of the dataset
        avg_loss = loss_train * 2 / dataset_sizes[TRAIN]
        avg_acc = acc_train * 2 / dataset_sizes[TRAIN]
        
        # For validation

        vgg.train(False)
        vgg.eval()
            
        for i, data in enumerate(dataloaders[VAL]):
            if i % 100 == 0:
                print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)
                
            inputs, labels = data
            
            if use_gpu:
                inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
            else:
                inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
            
            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss_val += loss.data
            acc_val += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()

        avg_loss_val = loss_val / dataset_sizes[VAL]
        avg_acc_val = acc_val / dataset_sizes[VAL]

        # For test
        vgg.train(False)
        vgg.eval()
            
        for i, data in enumerate(dataloaders[TEST]):
            if i % 100 == 0:
                print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)
                
            inputs, labels = data
            
            if use_gpu:
                inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
            else:
                inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
            
            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss_test += loss.data
            acc_test += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        avg_loss_test = loss_test / dataset_sizes[TEST]
        avg_acc_test = acc_test / dataset_sizes[TEST]
        
        print()
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        accuracy_train_list.append(avg_acc)
        loss_train_list.append(avg_loss)
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        accuracy_val_list.append(avg_acc_val)
        loss_val_list.append(avg_loss_val)
        print("Avg loss (test): {:.4f}".format(avg_loss_test))
        print("Avg acc (test): {:.4f}".format(avg_acc_test))
        accuracy_test_list.append(avg_acc_test)
        loss_test_list.append(avg_loss_test)
        print('-' * 10)
        print()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(vgg.state_dict())
        
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    vgg.load_state_dict(best_model_wts)
    return vgg, accuracy_test_list, loss_test_list, accuracy_train_list, loss_train_list,accuracy_val_list, loss_val_list,num_epoch_list

In [ ]:
vgg16, accuracy_test, loss_test, accuracy_train, loss_train,accuracy_val, loss_val,num_epoch = train_model(vgg16, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=1)
# torch.save(vgg16.state_dict(), 'VGG16_v2-OCT_Retina_half_dataset.pt')

Epoch 0/1
----------
Training batch 0/42.5OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUTS OF VGG
torch.Size([8, 2])
OUTPUT

KeyboardInterrupt: ignored

In [ ]:
plt.plot(num_epoch,accuracy_test, label='Test')
plt.plot(num_epoch,accuracy_train, label='Train')
plt.plot(num_epoch,accuracy_val, label='Val')
plt.legend()
plt.title("Epoch vs Accuracy")
plt.xlabel("No. of Epochs")
plt.ylabel("Accuracy")
plt.show()
plt.plot(num_epoch,loss_test, label='Test')
plt.plot(num_epoch,loss_train, label='Train')
plt.plot(num_epoch,loss_val, label='Val')
plt.legend()
plt.title("Epoch vs Loss")
plt.xlabel("No. of Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
eval_model(vgg16, criterion)

In [ ]:
visualize_model(vgg16, num_images=32)